In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
import nbimporter
from preprocessing import spectral_entropy, extract_features_task

In [ ]:
def preprocess_extract_data(path, subject, result_file):
    data = []
    tasks = sorted([f for f in os.listdir(path) if f.endswith('.csv')])

    result_df = pd.read_csv(result_file)
    subject_row = result_df[result_df['Subject'] == subject]

    addiction_level = subject_row['Internet Addiction Result'].values[0]
    personality = subject_row['Personality Result'].values[0]
    gender = subject_row['Gender'].values[0]
    memory_words_correct = subject_row['Memory Words Correct'].values[0]

    subject_features = []

    for task in tasks:
        task_file_path = os.path.join(path, task)

        try:
            column_names = ['attention', 'meditation', 'LowAlpha', 'HighAlpha', 'LowBeta', 'HighBeta', 'Delta', 'Theta', 'LowGamma', 'MiddleGamma']
            df = pd.read_csv(task_file_path, header=None, names=column_names)
            task_features = extract_features_task(df)
            subject_features.extend(task_features)
        except Exception as e:
            print(f"Skipping {task_file_path}: {e}")
            subject_features.extend([0] * 68)

    subject_features.extend([gender, memory_words_correct, addiction_level, personality])
    data.append(subject_features)

    df = pd.DataFrame(data)

    am_features = ['attention_mean', 'attention_std', 'meditation_mean', 'meditation_std']
    bands = ['LowAlpha', 'HighAlpha', 'LowBeta', 'HighBeta', 'Delta', 'Theta', 'LowGamma', 'MiddleGamma']
    eeg_stats = ['mean', 'std', 'kurtosis', 'skewness', 'range', 'bandpower_sum', 'spectral_entropy']
    eeg_features = [f"{band}_{stat}" for band in bands for stat in eeg_stats]
    features_per_task = am_features + eeg_features

    all_feature_names = []

    for task_idx in range(1, 12):
        all_feature_names.extend([f"task{task_idx}_{feat}" for feat in features_per_task])

    all_feature_names += ['gender', 'memory_words_correct', 'internet_addiction_level', 'personality']

    df.columns = all_feature_names

    return df


In [ ]:
def predict_Internet_addiction(df, le_gender, le_internet_addiction, pca, sc, models, num_classes = 3):
    df['gender_encoded'] = le_gender.transform(df['gender'])
    df['internet_addiction_level_encoded'] = le_internet_addiction.transform(df['internet_addiction_level'].replace({'Moderate level': 'Severe Level'}))

    X = df.drop(columns = ['gender', 'internet_addiction_level', 'personality', 'internet_addiction_level_encoded'])
    y = df['internet_addiction_level_encoded']

    X_scaled = sc.transform(X)
    X_pca = pca.transform(X_scaled)

    probs = [0] * num_classes
    for model in models:
        probs += model.predict_proba(X_pca)

    return np.argmax(probs / float(len(models)), axis = 1)
    

In [ ]:
import joblib

le_gender = joblib.load('inference/le_gender.pkl')
le_internet_addiction = joblib.load('inference/le_internet_addiction.pkl')
pca = joblib.load('inference/pca.pkl')
sc = joblib.load('inference/sc.pkl')

svm = joblib.load('best_model_svm.pkl')
rf = joblib.load('best_rf.pkl')
xgb = joblib.load('best_xgb.pkl')

## Test Subjects (excluded from training)

- **S1  : Normal Level**
- **S9  : Mid Level**
- **S13 : Mid Level**
- **S15 : Severe Level**
- **S21 : Normal Level**
- **S26 : Severe Level**


In [ ]:
subject = 'S21'

In [ ]:
data_path = 'Electroencephalogram (EEG) dataset with internet addiction and healthy teenagers under rest and executive function task/Data/' + subject
result_file_path = 'Electroencephalogram (EEG) dataset with internet addiction and healthy teenagers under rest and executive function task/Data/ResultCombine.csv'

In [ ]:
df = preprocess_extract_data(data_path, subject, result_file_path)
y_pred = predict_Internet_addiction(df, le_gender, le_internet_addiction, pca, sc, [svm, xgb, rf])

In [ ]:
print("Y Predicted: ", le_internet_addiction.inverse_transform(y_pred[0].reshape(-1))[0])